In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [3]:
# data preprocessing
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

cols_to_drop = ["id", "date", "zipcode"]

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df = test_df.drop(columns=cols_to_drop, errors='ignore')

# scale price
train_df["price"] = train_df["price"] / 1000
test_df["price"] = test_df["price"] / 1000

X_train = train_df.drop(columns=["price"])
y_train = train_df["price"]

X_test = test_df.drop(columns=["price"])
y_test = test_df["price"]

# scale each feature so that the mean is 0, and the standard deviation is 1
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [4]:
# training mlr
model = LinearRegression()
model.fit(X_train_scaled, y_train)

coef_table = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_
})

print("\nFeature Coefficients:")
print(coef_table)




Feature Coefficients:
          Feature  Coefficient
0      Unnamed: 0     8.456024
1        bedrooms   -12.807339
2       bathrooms    18.456913
3     sqft_living    57.161582
4        sqft_lot    11.127338
5          floors     8.151038
6      waterfront    64.230911
7            view    47.610288
8       condition    12.647609
9           grade    92.511076
10     sqft_above    48.439051
11  sqft_basement    27.688812
12       yr_built   -68.043173
13   yr_renovated    17.341926
14            lat    78.129852
15           long    -1.437669
16  sqft_living15    45.479128
17     sqft_lot15   -12.906560


In [5]:
y_train_pred = model.predict(X_train_scaled)

train_mse = mean_squared_error(y_train, y_train_pred)
print("\nTraining MSE:", train_mse)


Training MSE: 31415.747916100867


In [6]:
train_r2 = r2_score(y_train, y_train_pred)
print("Training R^2:", train_r2)

Training R^2: 0.7271450489303788


In [7]:
y_test_pred = model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\nTesting MSE:", test_mse)
print("Testing R^2:", test_r2)


Testing MSE: 58834.673978213985
Testing R^2: 0.6471195893437872


In [8]:
# add column of ones to design matrix 
X_train_np = X_train_scaled
y_train_np = y_train.values.reshape(-1, 1)
X_train_aug = np.hstack([np.ones((X_train_np.shape[0], 1)), X_train_np])

# closed form solution
theta = np.linalg.inv(X_train_aug.T @ X_train_aug) @ X_train_aug.T @ y_train_np


In [9]:
def predict(X_new_scaled, theta):
    """
    X_new_scaled : numpy array of shape (n_samples, n_features)
    theta        : numpy array of shape (n_features+1, 1)
    """
    # augment with col of 1s
    X_new_aug = np.hstack([np.ones((X_new_scaled.shape[0], 1)), X_new_scaled])
    return X_new_aug @ theta 


In [10]:
y_train_pred_manual = predict(X_train_scaled, theta)
y_test_pred_manual = predict(X_test_scaled, theta)

In [11]:
# compute metrics
train_mse_manual = mean_squared_error(y_train, y_train_pred_manual)
train_r2_manual = r2_score(y_train, y_train_pred_manual)

test_mse_manual = mean_squared_error(y_test, y_test_pred_manual)
test_r2_manual = r2_score(y_test, y_test_pred_manual)

print("Manual Linear Regression - Training MSE:", train_mse_manual)
print("Manual Linear Regression - Training R^2:", train_r2_manual)
print("Manual Linear Regression - Testing MSE:", test_mse_manual)
print("Manual Linear Regression - Testing R^2:", test_r2_manual)

Manual Linear Regression - Training MSE: 36000.67530645922
Manual Linear Regression - Training R^2: 0.6873236147218099
Manual Linear Regression - Testing MSE: 62811.14764798538
Manual Linear Regression - Testing R^2: 0.6232693737027156


In [18]:
def polynomial_features(X, degree):
    X = np.asarray(X).reshape(-1, 1)
    return np.hstack([X**d for d in range(1, degree + 1)])


In [19]:
def fit_polynomial_regression(X, y, degree):
    X_poly = polynomial_features(X, degree)
    X_augmented = np.hstack([np.ones((X_poly.shape[0], 1)), X_poly])
    y = y.reshape(-1, 1)
    theta = np.linalg.inv(X_augmented.T @ X_augmented) @ X_augmented.T @ y
    return theta 

In [20]:
def predict_polynomial(X, theta, degree):
    
    X_poly = polynomial_features(X, degree)
    X_aug = np.hstack([np.ones((X_poly.shape[0], 1)), X_poly])
    
    return X_aug @ theta


In [21]:
def fit_closed_form(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X] 
    theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
    return theta
def predict_closed_form(X, theta):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b @ theta

In [22]:
X_train_sqft = X_train[['sqft_living']].values
X_test_sqft  = X_test[['sqft_living']].values


In [ ]:
results = []

degrees = [1, 2, 3, 5]

for p in degrees:
    
    X_train_poly = polynomial_features(X_train_sqft, p)
    X_test_poly  = polynomial_features(X_test_sqft, p)
    
    theta = fit_closed_form(X_train_poly, y_train)
    
    y_train_pred = predict_closed_form(X_train_poly, theta)
    y_test_pred  = predict_closed_form(X_test_poly, theta)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse  = mean_squared_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2  = r2_score(y_test, y_test_pred)
    
    results.append([p, train_mse, test_mse, train_r2, test_r2])


Train poly shape: (1000, 1)
Train poly shape: (1000, 2)
Train poly shape: (1000, 3)
Train poly shape: (1000, 5)


In [24]:
import pandas as pd

results_df = pd.DataFrame(
    results,
    columns=["Degree p", "Train MSE", "Test MSE", "Train R²", "Test R²"]
)

print(results_df)


   Degree p     Train MSE      Test MSE  Train R²     Test R²
0         1  57947.526161  8.857598e+04  0.496709    0.468736
1         2  54822.665116  7.179168e+04  0.523849    0.569406
2         3  53785.194716  9.983348e+04  0.532860    0.401216
3         5  52626.111955  2.865728e+07  0.542927 -170.881541


# Problem 5

In [30]:
def compute_cost(X, y, theta):
    m = len(y)
    predictions = X @ theta
    error = predictions - y
    return (1 / (2 * m)) * np.sum(error**2)

In [31]:
def gradient_descent(X, y, theta, alpha = 0.01, num_iters = 1000):
    m = len(y)
    cost_history = []
    for _ in range(num_iters):
        predictions = X @ theta
        gradient = (1/m) * (X.T @ (predictions - y))
        theta = theta - alpha * gradient

        cost_history.append(compute_cost(X, y, theta))
    return theta, cost_history

def add_intercept(X):
    return np.c_[np.ones((X.shape[0], 1)), X]


In [32]:
X_train_gd = add_intercept(X_train_scaled)

theta_init = np.zeros(X_train_gd.shape[1])

theta_gd, cost_history = gradient_descent(
    X_train_gd,
    y_train,
    theta_init,
    alpha=0.01,
    num_iters=5000
)

print("Learned parameters:", theta_gd)

def predict_gd(X, theta):
    X = add_intercept(X)
    return X @ theta

Learned parameters: [520.414834     8.45596481 -12.80702978  18.45703883  57.16053175
  11.12163759   8.15140424  64.23053237  47.60980377  12.64760666
  92.50946386  48.43786851  27.6888169  -68.04271631  17.34219195
  78.13001983  -1.43845917  45.48258088 -12.90078962]


In [36]:
X_train_gd = add_intercept(X_train_scaled)
X_test_gd  = add_intercept(X_test_scaled)

theta_init = np.zeros(X_train_gd.shape[1])


learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]

results = []

for alpha in learning_rates:
    for iters in iterations_list:
        
        theta, _ = gradient_descent(
            X_train_gd,
            y_train,
            theta_init.copy(),
            alpha,
            iters
        )
        y_train_pred = X_train_gd @ theta
        y_test_pred  = X_test_gd @ theta
    
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse  = mean_squared_error(y_test, y_test_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2  = r2_score(y_test, y_test_pred)
        
        results.append({
            "alpha": alpha,
            "iterations": iters,
            "theta_norm": np.linalg.norm(theta),
            "train_MSE": train_mse,
            "test_MSE": test_mse,
            "train_R2": train_r2,
            "test_R2": test_r2
        })

results_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.4f}'.format)
print(results_df)

results_df.to_csv("gradient_descent_results.csv", index=False)


   alpha  iterations                   theta_norm  \
0 0.0100          10                      67.6553   
1 0.0100          50                     238.4885   
2 0.0100         100                     362.3810   
3 0.1000          10                     371.7395   
4 0.1000          50                     548.9296   
5 0.1000         100                     552.7541   
6 0.5000          10                   10793.9108   
7 0.5000          50           1810779882738.0154   
8 0.5000         100 34246110495022262517760.0000   

                                           train_MSE  \
0                                        294796.6946   
1                                        138298.5227   
2                                         70094.3812   
3                                         66473.6109   
4                                         31510.7162   
5                                         31427.5040   
6                                     616361041.0041   
7                    

# Problem 6

In [37]:
def ridge_gradient_descent(X, y, alpha=0.01, iterations=100, lam=0.1):
    N, d = X.shape
    theta = np.zeros(d)
    
    for _ in range(iterations):
        predictions = X @ theta
        errors = predictions - y
        
        gradient = (2/N) * (X.T @ errors)
        
        reg = 2 * lam * theta
        
        reg[0] = 0
        
        theta = theta - alpha * (gradient + reg)
    
    return theta


In [38]:
X_train_gd = add_intercept(X_train_scaled)
X_test_gd  = add_intercept(X_test_scaled)

theta_init = np.zeros(X_train_gd.shape[1])

learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]
lambdas = [0.01, 0.1, 1.0]  

results = []

for lam in lambdas:
    for alpha in learning_rates:
        for iters in iterations_list:
            
            theta = ridge_gradient_descent(
                X_train_gd,
                y_train,
                alpha=alpha,
                iterations=iters,
                lam=lam
            )
            
            y_train_pred = X_train_gd @ theta
            y_test_pred  = X_test_gd @ theta
        
            train_mse = mean_squared_error(y_train, y_train_pred)
            test_mse  = mean_squared_error(y_test, y_test_pred)
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2  = r2_score(y_test, y_test_pred)
            
            results.append({
                "lambda": lam,
                "alpha": alpha,
                "iterations": iters,
                "theta_norm": np.linalg.norm(theta),
                "train_MSE": train_mse,
                "test_MSE": test_mse,
                "train_R2": train_r2,
                "test_R2": test_r2
            })

results_df = pd.DataFrame(results)

pd.set_option('display.float_format', '{:.6f}'.format)
print(results_df)

results_df.to_csv("ridge_gradient_descent_results.csv", index=False)


     lambda    alpha  iterations  \
0  0.010000 0.010000          10   
1  0.010000 0.010000          50   
2  0.010000 0.010000         100   
3  0.010000 0.100000          10   
4  0.010000 0.100000          50   
5  0.010000 0.100000         100   
6  0.010000 0.500000          10   
7  0.010000 0.500000          50   
8  0.010000 0.500000         100   
9  0.100000 0.010000          10   
10 0.100000 0.010000          50   
11 0.100000 0.010000         100   
12 0.100000 0.100000          10   
13 0.100000 0.100000          50   
14 0.100000 0.100000         100   
15 0.100000 0.500000          10   
16 0.100000 0.500000          50   
17 0.100000 0.500000         100   
18 1.000000 0.010000          10   
19 1.000000 0.010000          50   
20 1.000000 0.010000         100   
21 1.000000 0.100000          10   
22 1.000000 0.100000          50   
23 1.000000 0.100000         100   
24 1.000000 0.500000          10   
25 1.000000 0.500000          50   
26 1.000000 0.500000        

In [39]:
np.random.seed(42)

N = 1000

X = np.random.uniform(-2, 2, N)
epsilon = np.random.normal(0, np.sqrt(2), N)

y = 1 + 2*X + epsilon
X_design = np.c_[np.ones(N), X]  



In [40]:
theta_ols = np.linalg.inv(X_design.T @ X_design) @ X_design.T @ y
y_pred_ols = X_design @ theta_ols

ols_mse = mean_squared_error(y, y_pred_ols)
ols_r2 = r2_score(y, y_pred_ols)


In [41]:
def ridge_closed_form(X, y, lam):
    n_features = X.shape[1]
    
    I = np.eye(n_features)
    I[0,0] = 0 
    
    return np.linalg.inv(X.T @ X + lam*I) @ X.T @ y


In [42]:
lambdas = [1, 10, 100, 1000, 10000]

results = []

results.append({
    "model": "OLS",
    "lambda": 0,
    "intercept": theta_ols[0],
    "slope": theta_ols[1],
    "MSE": ols_mse,
    "R2": ols_r2
})

for lam in lambdas:
    theta_ridge = ridge_closed_form(X_design, y, lam)
    y_pred = X_design @ theta_ridge
    
    results.append({
        "model": "Ridge",
        "lambda": lam,
        "intercept": theta_ridge[0],
        "slope": theta_ridge[1],
        "MSE": mean_squared_error(y, y_pred),
        "R2": r2_score(y, y_pred)
    })

results_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.6f}'.format)
print(results_df)


   model  lambda  intercept    slope      MSE       R2
0    OLS       0   1.137727 1.945275 1.949936 0.725824
1  Ridge       1   1.137671 1.943850 1.949939 0.725823
2  Ridge      10   1.137175 1.931119 1.950209 0.725785
3  Ridge     100   1.132549 1.812414 1.974016 0.722438
4  Ridge    1000   1.105658 1.122450 2.873516 0.595961
5  Ridge   10000   1.071013 0.233509 5.947067 0.163796
